# Evaluate JSON output(s)

In [1]:
import json
import pandas as pd
from os import listdir
from os.path import isfile, join

In [2]:
PATH_OUTPUT = "output/"
output_files = [f for f in listdir(PATH_OUTPUT) if isfile(join(PATH_OUTPUT, f)) and "schema" not in f]

# assume ([\d_]_)+_output.json format
output_files.sort(key=lambda x:int(x.replace("pp","").rstrip("output.json").strip("_")))

output = []
for f in output_files:
    d = {}
    with open(join(PATH_OUTPUT,f), "r") as file:
        data = json.load(file)
        file_id = f.rstrip("output.json").strip("_")
        d["ID"] = file_id
        d["no_regests"] = len(data["proceedings"])
        people_data=0
        empty_error_tags=0
        for proceeding in data["proceedings"]:
            p = proceeding["proceeding"]
            m = proceeding["meta"]
            # check if proceeding contains information about people
            if p["people"]:
                people_data+=1
            # check if meta contains no error tags
            if not m["error_tags"]:
                empty_error_tags+=1
        d["people_data"] = people_data
        d["empty_error_tags"] = empty_error_tags
        error_tags = {}
        for proceeding in data["proceedings"]:
            m = proceeding["meta"]
            for error_tag in m["error_tags"]:
                if error_tag in error_tags:
                    error_tags[error_tag]+=1
                else:
                    error_tags[error_tag]=1
        error_tags_sorted = sorted([(k,v) for (k,v) in error_tags.items()],key=lambda x:x[1],reverse=True)
        for et in error_tags_sorted:
            d[et[0]]=et[1]
    output.append(d)
df = pd.DataFrame(output)
df.transpose()

,0,1,2,3
ID,2022,2023_01,2023_02,2023_02_pp
no_regests,9562,9692,9634,9634
people_data,7460,7874,7888,7888
empty_error_tags,5081,5481,5601,5764
missing_accusation,2770,2516,2274,2274
missing_result,2717,2454,2178,2178
missing_people,2102,1818,1746,1746
missing_ID,1411,1010,953,677
missing_shelfmark,1352,905,848,612
missing_date_of_birth,1304,1269,1297,1355
